In [ ]:
from src.projects.fagradalsfjall.common.model_eval import ModelEvalResult, ValidationType
from src.projects.fagradalsfjall.common.model_eval_plot_curves import generate_all_rmse_plots
from src.projects.fagradalsfjall.common.model_eval_plot_simulations import plot_simulations
from src.projects.fagradalsfjall.common.model_repo import ModelRepo
from src.projects.fagradalsfjall.common.paths import get_blog_post_subfolder
from src.projects.fagradalsfjall.common.project_settings import TS_PRIMARY_METRIC_DISPLAY_NAME
from src.tools.datetime import format_timedelta

# 2. Visualization

In [ ]:
# -------------------------------------------------------------------------
#  Output path settings
# -------------------------------------------------------------------------
path_results_mlp_auc = get_blog_post_subfolder(6, "results_2_mlp_auc")

In [ ]:
# -------------------------------------------------------------------------
#  Load data
# -------------------------------------------------------------------------

# --- load models -------------------------------------
prev_model_eval_results = ModelRepo.load_models(["naive-mean", "naive-constant", "naive-exp-decay", "linear-ar"])

mlp_auc_model_eval_results = ModelRepo.load_models(["mlp-auc"])
mlp_mse_n_1_model_eval_results = ModelRepo.load_models(["mlp-rmse-n-1"])
mlp_mse_n_192_model_eval_results = ModelRepo.load_models(["mlp-rmse-n-192"])

# rename
mlp_mse_n_1_model_eval_results = {"mlp-mse-n-1": mlp_mse_n_1_model_eval_results["mlp-rmse-n-1"]}
mlp_mse_n_192_model_eval_results = {"mlp-mse-n-192": mlp_mse_n_192_model_eval_results["mlp-rmse-n-192"]}

# --- post-processing ---------------------------------
model_mlp_auc = mlp_auc_model_eval_results["mlp-auc"].ts_model

In [ ]:
# -------------------------------------------------------------------------
#  Overall grid search 1D results
# -------------------------------------------------------------------------
param_names = ["n_epochs", "n_hidden_layers", "dropout", "n"]

for param_name in param_names:

    fig, ax = (
        model_mlp_auc.cv.results.plot_1d(param_names=param_name)
        .set_fig_title(f"Cross-validation - 'mlp-auc' - best results for each value of '{param_name}'")
        .set_y_label(TS_PRIMARY_METRIC_DISPLAY_NAME)
        .set_x_label(param_name)
        .create(w=12, h=8)
    )
    fig.savefig(path_results_mlp_auc / f"cv_1d_best_{param_name}.png", dpi=600)

In [ ]:
# -------------------------------------------------------------------------
#  RMSE curves
# -------------------------------------------------------------------------

# merge all evaluation results in single dict for visualization
model_eval_results = (
    prev_model_eval_results
    | mlp_mse_n_1_model_eval_results
    | mlp_mse_n_192_model_eval_results
    | mlp_auc_model_eval_results
)

# create RMSE plots
generate_all_rmse_plots(model_eval_results, path_results_mlp_auc, highlight_models=["mlp-auc"])